In [ ]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from torch.utils.data import DataLoader

# Define TrajectoryDataset class
class TrajectoryDataset(Dataset):
    def __init__(self, dataframe, window_length=100):
        # Normalize the data
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        normalized_df = pd.DataFrame(self.scaler.fit_transform(dataframe), columns=dataframe.columns)

        # Apply custom transformation for windowing
        sliced_df = self.custom_transformation(normalized_df.to_numpy(), window_length=window_length)

        # Convert to tensor for PyTorch
        self.data = torch.tensor(sliced_df, dtype=torch.float32)

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data[idx]

    def custom_transformation(self, dataframe_array, window_length):
        # Windowing to create inputs and targets
        window_length += 1  # Extra column for the target
        sliced_data = np.lib.stride_tricks.sliding_window_view(dataframe_array, window_shape=(window_length,), axis=1)
        sliced_data = sliced_data.reshape(-1, window_length)
        return sliced_data

    def inverse_transform(self, data):
        # Apply inverse transformation to get back original scale
        return self.scaler.inverse_transform(data)


In [ ]:
#LSTM Model

import torch.nn as nn

# Define the LSTM-based model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Fully connected layer to produce final output
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])  # Last time step output
        return out


In [ ]:
import torch

def autoregressive_predict(model, initial_input, prediction_length, device):
    """
    Perform autoregressive prediction using the trained LSTM model.
    
    Args:
    - model: The trained PyTorch model.
    - initial_input: The last sequence of time steps used as input for prediction, shaped [1, sequence_length, input_size].
    - prediction_length: The number of future time steps to predict.
    - device: The device (CPU or GPU) for computation.
    
    Returns:
    - predictions: A list of predicted values over the prediction_length.
    """
    model.eval()  # Set model to evaluation mode
    predictions = []
    
    # Move initial input to the device
    current_input = initial_input.to(device)
    
    with torch.no_grad():  # Disable gradient calculation for prediction
        for _ in range(prediction_length):
            # Perform a forward pass to get the next prediction
            output = model(current_input)
            
            # Save the prediction (output is [1, output_size])
            next_value = output.item()  # Convert to scalar if output_size=1
            predictions.append(next_value)
            
            # Update current_input by removing the first time step and appending the prediction
            next_input = torch.cat((current_input[:, 1:, :], output.unsqueeze(0).unsqueeze(-1)), dim=1)
            current_input = next_input  # Update for the next prediction

    return predictions


In [ ]:
# Paths to the uploaded files
train_path = 'train.csv'
val_path = 'val.csv'
test_path = 'test.csv'

# Load the dataframes
train_df = pd.read_csv(train_path, header=0).drop('ids', axis=1, errors='ignore')
val_df = pd.read_csv(val_path, header=0).drop('ids', axis=1, errors='ignore')
test_df = pd.read_csv(test_path, header=0).drop('ids', axis=1, errors='ignore')

# Initialize dataset with normalization and windowing
window_length = 100  # Adjust as needed for your model
train_dataset = TrajectoryDataset(dataframe=train_df, window_length=window_length)
val_dataset = TrajectoryDataset(dataframe=val_df, window_length=window_length)
test_dataset = TrajectoryDataset(dataframe=test_df, window_length=window_length)

# Extract validation set data as a tensor from the val_dataset
val_set = torch.tensor(val_df.values[:,:].astype(np.float32), dtype=torch.float32)


In [ ]:
#hyper parameter tuning
from itertools import product

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define ranges for hyperparameters
hidden_sizes = [32, 64, 128]
num_layers_list = [1, 2, 3]
learning_rates = [0.001, 0.01]
batch_sizes = [8, 16, 32]

# Variable to track the best performance
best_mse = float('inf')
best_params = None

# Iterate over all combinations of hyperparameters
for hidden_size, num_layers, learning_rate, batch_size in product(hidden_sizes, num_layers_list, learning_rates, batch_sizes):
    # Initialize the model
    model = LSTMModel(input_size=1, hidden_size=hidden_size, num_layers=num_layers, output_size=1).to(device)
    
    # Define optimizer and loss function
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Create DataLoader for the training dataset
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Train the model for a few epochs (e.g., 3) for quick evaluation
    num_epochs_tuning = 3
    for epoch in range(num_epochs_tuning):
        model.train()
        running_loss = 0.0
        
        for i, data in enumerate(train_loader):
            inputs = data[:, :-1].reshape(-1, 100, 1).to(device)
            targets = data[:, -1].to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
    
    # Evaluate the model on the validation set
    model.eval()
    initial_input_val = val_set[:, -window_length:].reshape(1, -1, 1).to(device)
    full_trajectories = autoregressive_predict(model, initial_input_val, prediction_length=val_set.shape[1], device=device)
    
    # Calculate MSE for validation predictions
    mse = criterion(full_trajectories, val_set).item()
    
    # Check if this is the best combination
    if mse < best_mse:
        best_mse = mse
        best_params = (hidden_size, num_layers, learning_rate, batch_size)
        print(f"New best MSE: {best_mse:.4f} with params: hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, batch_size={batch_size}")
best_hidden_size=best_params[0]
best_num_layers=best_params[1]
best_learning_rate=best_params[2]
best_batch_size=best_params[3]
#Print the best hyperparameters found
print(f"Best parameters found: hidden_size={best_params[0]}, num_layers={best_params[1]}, learning_rate={best_params[2]}, batch_size={best_params[3]}")
torch.save(model.state_dict(), 'model_checkpoint.pth')


In [ ]:

# Model hyperparameters
# input_size = 1              # Each time step has a single feature (the value at that time step)
# hidden_size = 64            # Number of units in the LSTM layer
# num_layers = 2              # Number of LSTM layers
# output_size = 1             # Predicting the next value in sequence
num_epochs = 5             # Number of training epochs

# Re-initialize and train the model using the best hyperparameters
model = LSTMModel(input_size=1, hidden_size=best_hidden_size, num_layers=best_num_layers, output_size=1).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=best_learning_rate)
train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)

accumulation_steps = 4
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for i, data in enumerate(train_loader):
        inputs = data[:, :-1].reshape(-1, 100, 1).to(device)
        targets = data[:, -1].to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        
        # Only update weights every 'accumulation_steps' batches
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")
torch.save(model.state_dict(), 'model_checkpoint.pth')



In [ ]:
# Select the last 100 time steps from the validation set as the initial input
initial_input = val_dataset.data[-1, :-1].reshape(1, 100, 1)  # Shape to [1, 100, 1]

# Number of future steps to predict
prediction_length = 10  # Predict the next 10 time steps

# Perform autoregressive prediction
predicted_sequence = autoregressive_predict(model, initial_input, prediction_length, device)

# Display predicted sequence
print("Predicted future values:", predicted_sequence)
torch.save(model.state_dict(), 'model_checkpoint.pth')


In [ ]:
from torch.nn import MSELoss


train_set = torch.tensor(train_df.values[:,:].astype(np.float32), dtype=torch.float32)
val_set = torch.tensor(val_df.values[:,:].astype(np.float32), dtype=torch.float32)
test_set = torch.tensor(val_df.values[:,:].astype(np.float32), dtype=torch.float32)

points_to_predict = val_set.shape[1]

# Autoregressive prediction function
def autoregressive_predict(model, input_maxtrix, prediction_length=points_to_predict):
    """
    Perform autoregressive prediction using the learned model.

    Args:
    - model: The trained PyTorch model.
    - input_maxtrix: A matrix of initial time steps (e.g., shape (963, window_length)).
    - prediction_length: The length of the future trajectory to predict.

    Returns:
    - output_matrix: A tensor of the predicted future trajectory of the same length as `prediction_length`.
    """
    model.eval()  # Set model to evaluation mode
    output_matrix = torch.empty(input_maxtrix.shape[0],0)
    current_input = input_maxtrix

    with torch.no_grad():  # No need to calculate gradients for prediction
        for idx in range(prediction_length):
            # Predict the next time step
            next_pred = model(current_input)

            # Concatenating the new column along dimension 1 (columns)
            output_matrix = torch.cat((output_matrix, next_pred), dim=1)

            # Use the predicted value as part of the next input
            current_input = torch.cat((current_input[:, 1:],next_pred),dim=1)

    return output_matrix


initial_input = train_set[:, -window_length:]  #use the last window of training set as initial input
full_trajectories = autoregressive_predict(model, initial_input)


# Calculate MSE between predicted trajectories and actual validation trajectories using torch
mse_loss = MSELoss()

# Compute MSE
mse = mse_loss(full_trajectories, val_set)

# Print MSE
print(f'Autoregressive Validation MSE (using torch): {mse.item():.4f}')
torch.save(model.state_dict(), 'model_checkpoint.pth')


In [ ]:
# Perform autoregressive predictions for one row in the validation set
# We can pick a specific row (e.g., row 0) to visualize
row_idx = 0  # You can change this to visualize predictions for different rows
initial_input = val_set[row_idx, :window_length].unsqueeze(0)

# Predict future trajectory of length 100
predicted_trajectory = autoregressive_predict(model, initial_input)

# Get the actual trajectory for comparison
actual_trajectory = val_set[row_idx].numpy()

# Plot the actual vs predicted trajectory
plt.figure(figsize=(10, 6))
plt.plot(range(len(actual_trajectory)), actual_trajectory, label="Actual Trajectory", color='blue', marker='o')
plt.plot(range(len(actual_trajectory)), predicted_trajectory.squeeze().numpy(), label="Predicted Trajectory", color='red', linestyle='--', marker='x')
plt.title(f"Actual vs Predicted Trajectory (Row {row_idx})")
plt.xlabel("Time Step")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()
torch.save(model.state_dict(), 'model_checkpoint.pth')


In [ ]:
# Generate predictions for all the validation dataset
initial_input = train_set[:, -window_length:]
val_predictions_tensor = autoregressive_predict(model, initial_input)

# Generate predictions for all the test dataset
initial_input = val_predictions_tensor[:, -window_length:]
test_predictions_tensor = autoregressive_predict(model, initial_input)


# Print their shapes
print(f'Validation Predictions Tensor Shape: {val_predictions_tensor.shape}')
print(f'Test Predictions Tensor Shape: {test_predictions_tensor.shape}')
torch.save(model.state_dict(), 'model_checkpoint.pth')


In [ ]:
def generate_submissions_v4(pred_val_tensor, pred_test_tensor, original_val_path, original_test_path):
    # Read the original validation and testing datasets
    original_val_df = pd.read_csv(original_val_path)
    original_test_df = pd.read_csv(original_test_path)

    # Ensure the shape of pred_val_tensor and pred_test_tensor is correct
    assert pred_val_tensor.shape[0] * pred_val_tensor.shape[1] == original_val_df.shape[0] * (original_val_df.shape[1] - 1)
    assert pred_test_tensor.shape[0] * pred_test_tensor.shape[1] == original_test_df.shape[0] * (original_test_df.shape[1] - 1)

    # Create empty lists to store ids and values
    ids = []
    values = []

    # Process validation set
    for col_idx, col in enumerate(original_val_df.columns[1:]):  # Skip the 'ids' column
        for row_idx, _ in enumerate(original_val_df[col]):
            ids.append(str(f"{col}_traffic_val_{row_idx}"))
            values.append(float(pred_val_tensor[row_idx, col_idx]))

    # Process testing set
    for col_idx, col in enumerate(original_test_df.columns[1:]):  # Skip the 'ids' column
        for row_idx, _ in enumerate(original_test_df[col]):
            ids.append(str(f"{col}_traffic_test_{row_idx}"))
            values.append(float(pred_test_tensor[row_idx, col_idx]))

    # Create the submissions dataframe
    submissions_df = pd.DataFrame({
        "ids": ids,
        "value": values
    })

    # Impute any null values
    submissions_df.fillna(100, inplace=True)

    # Assert the shape of the dataframe
    assert submissions_df.shape[1] == 2
    assert submissions_df.shape[0] == (original_val_df.shape[0] * (original_val_df.shape[1] - 1)) + (original_test_df.shape[0] * (original_test_df.shape[1] - 1))
    assert "ids" in submissions_df.columns
    assert "value" in submissions_df.columns

    # Save to CSV
    submissions_df.to_csv('submissions_v3.csv', index=False)

# Call the function
generate_submissions_v4(val_predictions_tensor, test_predictions_tensor, '/kaggle/input/cse-575-project-2/val.csv', '/kaggle/input/cse-575-project-2/test.csv')
torch.save(model.state_dict(), 'model_checkpoint.pth')
